In [1]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import random
import math
from collections import defaultdict
import os
from scipy.spatial import Delaunay as SciPyDelaunay

EPSILON = 1e-9
ROT_ANGLE = 1e-4
PERTURB = 1e-7

def orient(a, b, c):
    return (b.x - a.x) * (c.y - a.y) - (b.y - a.y) * (c.x - a.x)

class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y
    def rotate(self, a):
        c, s = math.cos(a), math.sin(a)
        return Point(self.x * c - self.y * s, self.x * s + self.y * c)
    def perturbed(self):
        return Point(
            self.x + random.uniform(-PERTURB, PERTURB),
            self.y + random.uniform(-PERTURB, PERTURB)
        )
    def to_tuple(self):
        return (self.x, self.y)
    def __repr__(self):
        return f"({self.x:.4f},{self.y:.4f})"

class IncrementalConvexHull:
    def __init__(self, points):
        self.points = [Point(*p) if not isinstance(p, Point) else p for p in points]
        self.hull = []
        self.tri = []

    def preprocess(self):
        if len({p.x for p in self.points}) != len(self.points):
            self.points = [p.rotate(ROT_ANGLE) for p in self.points]
            rotated = True
        else:
            rotated = False
        self.points.sort(key=lambda p: (p.x, p.y))
        if rotated:
            self.points = [p.rotate(-ROT_ANGLE) for p in self.points]

    def init_hull(self):
        col = [self.points[0]]
        for pt in self.points[1:]:
            if len(col) == 1:
                col.append(pt)
                continue
            if abs(orient(col[-2], col[-1], pt)) < EPSILON:
                col.append(pt)
            else:
                break
        if len(col) == len(self.points):
            raise ValueError("Todos os pontos são colineares — sem triangulação 2D.")
        p_nc = self.points[len(col)]
        self.tri = []
        for i in range(len(col) - 1):
            a, b = col[i], col[i + 1]
            self.tri.append((p_nc, a, b))
        left, right = col[0], col[-1]
        if orient(left, right, p_nc) > 0:
            right, p_nc = p_nc, right
        self.hull = [left, right, p_nc]
        return len(col)

    def is_inside(self, p):
        for a, b in zip(self.hull, self.hull[1:] + self.hull[:1]):
            if orient(a, b, p) < -EPSILON:
                return False
        return True

    def add_point(self, p):
        if self.is_inside(p):
            return
        n = len(self.hull)
        visible = [i for i in range(n) if orient(self.hull[i], self.hull[(i + 1) % n], p) > EPSILON]
        for i in visible:
            a, b = self.hull[i], self.hull[(i + 1) % n]
            self.tri.append((a, b, p))
        if len(visible) > 1:
            for idx in visible[::-1][:-1]:
                del self.hull[idx]
        self.hull.insert(visible[0] + 1, p)

    def compute(self):
        self.preprocess()
        idx = self.init_hull()
        for p in self.points[idx + 1:]:
            self.add_point(p)
        return self.tri

def in_circle(a, b, c, d):
    """Retorna valor determinante para teste incircle em Delaunay."""
    ax, ay = a.x-d.x, a.y-d.y
    bx, by = b.x-d.x, b.y-d.y
    cx, cy = c.x-d.x, c.y-d.y
    return ((ax*ax+ay*ay)*(bx*cy-by*cx)
          - (bx*bx+by*by)*(ax*cy-ay*cx)
          + (cx*cx+cy*cy)*(ax*by-ay*bx))

class DelaunayTriangulation:
    def __init__(self, points, animated=False):
        self.points = [Point(*p) if not isinstance(p, Point) else p
                       for p in points]
        self.animated = animated
        self.triangles = []
        self.frames = []

    def compute(self):
        """Aplica flips de aresta até satisfazer a condição de Delaunay."""
        self.triangles = IncrementalConvexHull(self.points).compute()
        changed = True
        while changed:
            changed, edge_map = False, defaultdict(list)
            for tri in self.triangles:
                for u, v in [(tri[0],tri[1]),(tri[1],tri[2]),(tri[2],tri[0])]:
                    key = tuple(sorted((u,v), key=lambda p:(p.x,p.y)))
                    edge_map[key].append(tri)
            for (u, v), tris in edge_map.items():
                if len(tris)!=2: continue
                t1, t2 = tris
                w = (set(t1)-{u,v}).pop()
                z = (set(t2)-{u,v}).pop()
                if orient(w,u,z)*orient(w,v,z)>0: continue
                if orient(u,v,w)<0:
                    u, v = v, u; t1, t2 = t2, t1
                if in_circle(u,v,w,z)>0:
                    self.triangles.remove(t1)
                    self.triangles.remove(t2)
                    new1 = (w,z,u) if orient(w,z,u)>0 else (w,u,z)
                    new2 = (z,w,v) if orient(z,w,v)>0 else (z,v,w)
                    self.triangles.extend([new1,new2])
                    changed = True
                    if self.animated:
                        self.frames.append({'old':(u,v),'new':(w,z),
                                            'tr':list(self.triangles)})
                    break
        return self.triangles

    def animate(self, filename, fps=2, interval=500):
        """Salva vídeo do processo de flips mostrando cada troca de aresta."""
        fig, ax = plt.subplots(figsize=(6,6))
        def update(i):
            f = self.frames[i]
            ax.clear(); ax.set_title(f"Flip {i+1}: {f['old']}→{f['new']}")
            for tri in f['tr']:
                xs = [p.x for p in tri]+[tri[0].x]
                ys = [p.y for p in tri]+[tri[0].y]
                ax.plot(xs, ys, '-k')
            ax.scatter([p.x for p in self.points],[p.y for p in self.points])
            ax.set_aspect('equal'); ax.grid(True)
        anim = FuncAnimation(fig, update, frames=len(self.frames),
                             interval=interval, repeat=False)
        anim.save(filename, writer='ffmpeg', fps=fps)
        plt.close(fig)


def compare_with_scipy(points, our_tris):
    """Compara nossa triangulação com SciPy Delaunay e imprime resultado."""
    # Converte pontos para tuplas (caso sejam objetos Point)
    if isinstance(points[0], Point):
        point_tuples = [p.to_tuple() for p in points]
    else:
        point_tuples = [tuple(p) for p in points]  # Garante que tudo seja tupla

    arr = np.array(point_tuples)
    tri = SciPyDelaunay(arr)
    
    # Índices dos triângulos do SciPy
    scipy_set = {tuple(sorted(simplex)) for simplex in tri.simplices}
    
    # Índices dos nossos triângulos
    our_set = {tuple(sorted((point_tuples.index(a.to_tuple() if isinstance(a, Point) else a),
                             point_tuples.index(b.to_tuple() if isinstance(b, Point) else b),
                             point_tuples.index(c.to_tuple() if isinstance(c, Point) else c))))
               for a, b, c in our_tris}
    
    print(f"nosso={len(our_set)}; scipy={len(scipy_set)}")
    matches = len(our_set & scipy_set)
    print(f"iguais: {matches} de {len(scipy_set)}")


# Para testar o algoritmo, mude a váriavel "pts" abaixo e rode o algoritmo. 
Se quiser uma simulação visual com o matplotlib, mude a variável "animated" para "True". Se quiser gerar pontos aleatórios, utilize a função "generate_random_points" como pode ser visto abaixo. Caso contrário, passe uma array de pontos no formato [(a,b), (c,d), (e,f), ... (y,z)]. Se quiser ver apenas um plot da triangulação no final mude final_plot para True

In [2]:
random.seed(42)
def generate_random_points(num_points):
    return [(random.uniform(-20, 20), random.uniform(-20,20)) for _ in range(num_points)]

In [3]:
# MUDE APENAS ESSAS VARIÁVEIS
pts = generate_random_points(100)
animated=True
#A animação só funciona se a triangulação inicial(incremental) não for já a deulaney



if __name__ == "__main__":
    # Triangulação por flip-graph
    dt = DelaunayTriangulation(pts, animated=animated)
    our_tris = dt.compute()
    print("Triangulação Deulanay encontrada:", our_tris)
    # Compara contra SciPy
    #compare_with_scipy(pts, our_tris)
    # Salva animação dos flips
    os.makedirs("simulations", exist_ok=True)
    if animated:    
        dt.animate("simulations/delaunay_flip.mp4")


Triangulação Deulanay encontrada: [((-11.5607,17.7164), (-10.8423,16.2168), (-10.0077,16.9306)), ((-11.5607,17.7164), (-10.0077,16.9306), (-9.3209,17.4662)), ((-10.0077,16.9306), (-9.3977,14.8973), (-9.3209,17.4662)), ((-7.6994,-17.6830), (-10.8381,-18.7160), (-3.1231,-18.8081)), ((-11.5607,17.7164), (-9.3209,17.4662), (-1.8511,18.1526)), ((-9.3209,17.4662), (-2.2748,14.4540), (-1.8511,18.1526)), ((-19.2209,17.1639), (-11.5607,17.7164), (1.4491,18.9246)), ((-11.5607,17.7164), (-1.8511,18.1526), (1.4491,18.9246)), ((0.0234,-12.8539), (-3.1231,-18.8081), (2.0130,-17.9765)), ((4.3588,-13.8864), (2.0130,-17.9765), (5.5771,-18.9996)), ((2.0130,-17.9765), (-3.1231,-18.8081), (5.5771,-18.9996)), ((10.6334,-14.8643), (8.1829,-18.1670), (12.3772,-19.7400)), ((8.1829,-18.1670), (5.5771,-18.9996), (12.3772,-19.7400)), ((14.3854,-17.1657), (12.3772,-19.7400), (15.6872,-16.5224)), ((15.1204,17.8780), (16.5051,14.8207), (18.8431,14.4312)), ((15.6872,-16.5224), (12.3772,-19.7400), (19.3666,-16.0633))